In [3]:
import requests
from bs4 import BeautifulSoup

def crawl_vnexpress_articles(num_articles=50):
    url = "https://vnexpress.net"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    articles = []
    links = []

    # Lấy các thẻ chứa đường link bài viết chính
    for a in soup.select("a[href^='https://vnexpress.net']"):
        href = a.get("href")
        if href and href not in links and len(links) < num_articles:
            links.append(href)

    print(f"Đã thu thập {len(links)} link bài viết.")

    for link in links:
        try:
            r = requests.get(link, timeout=5)
            s = BeautifulSoup(r.text, "html.parser")

            # Lấy nội dung chính (thẻ article)
            body = s.find("article")
            if body:
                paragraphs = body.find_all("p")
                content = "\n".join([p.get_text() for p in paragraphs])
                if len(content) > 100:
                    articles.append(content)
        except:
            continue

    # Gộp toàn bộ nội dung và lưu
    full_text = "\n".join(articles)
    with open("raw_articles.txt", "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"Đã lưu {len(articles)} bài viết vào file raw_articles.txt")

# Gọi hàm crawl
crawl_vnexpress_articles()

Đã thu thập 50 link bài viết.
Đã lưu 50 bài viết vào file raw_articles.txt


In [4]:
import re
from underthesea import word_tokenize
import os

# 1. Đưa về chữ thường, loại bỏ dấu câu, số, ký tự đặc biệt, loại bỏ khoảng trắng dư thừa
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                  r'ùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 2. Đọc dữ liệu thô và làm sạch
with open('raw_articles.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
cleaned_text = clean_text(raw_text)

# 3. Tách từ bằng underthesea
tokenized_text = word_tokenize(cleaned_text, format="text")
tokens = tokenized_text.replace("_", " ").split()

# 4. Lọc từ tiếng Việt
def load_all_vietnamese_words(folder_path):
    all_words = set()
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    word = line.strip().lower()
                    if word:
                        all_words.add(word)
    return all_words

folder_path = 'tu_dien_goc'  # Thư mục chứa các file từ điển
vietnamese_dict = load_all_vietnamese_words(folder_path)
vietnamese_only_tokens = [word for word in tokens if word in vietnamese_dict]

# 5. Loại bỏ từ trùng lặp và sắp xếp theo thứ tự abc
unique_sorted = sorted(set(vietnamese_only_tokens), key=lambda x: x.lower())

# 6. Lưu kết quả ra file
with open('vietnamese_only_tokens_sorted.txt', 'w', encoding='utf-8') as f:
    for word in unique_sorted:
        f.write(word + '\n')

print(f"Đã lọc và sắp xếp {len(unique_sorted)} từ tiếng Việt duy nhất.")

Đã lọc và sắp xếp 1745 từ tiếng Việt duy nhất.


In [1]:
import random
import unicodedata

vowels = "aăâeêioôơuưy"
tone_marks = ["́", "̀", "̉", "̃", "̣"]

def remove_tone(char):
    decomposed = unicodedata.normalize('NFD', char)
    base = ''.join([c for c in decomposed if c not in tone_marks])
    return unicodedata.normalize('NFC', base)

def random_tone(char):
    if char not in vowels:
        return char
    base = unicodedata.normalize('NFD', char)[0]
    tone = random.choice(tone_marks)
    return unicodedata.normalize('NFC', base + tone)

def confused_initials(word):
    # Các nhóm phụ âm đầu dễ nhầm lẫn
    confusion_groups = [
        ["ch", "tr"],
        ["l", "n"],
        ["d", "r", "gi"],
        ["s", "x"],
        ["c", "k"],
        ['ngh', 'ng',"gh"],
    ]
    results = []
    for group in confusion_groups:
        for prefix in group:
            if word.startswith(prefix):
                for alt in group:
                    if alt != prefix:
                        results.append(alt + word[len(prefix):])
                return results  # chỉ đổi 1 nhóm đầu tiên tìm thấy
    return []

def generate_typos(word, num_typos=10):
    typo_list = []
    operations = [
        "repeat",      # Thừa chữ
        "delete",      # Thiếu chữ
        "remove_tone", # Thiếu dấu
        "wrong_tone",  # Sai dấu
        "transpose",   # Sai thứ tự chữ
        "confused_initial", # Nhầm phụ âm đầu
    ]
    for _ in range(num_typos):
        op = random.choice(operations)
        typo = None
        if op == "repeat" and len(word) > 0:
            pos = random.randint(0, len(word) - 1)
            typo = word[:pos] + word[pos] + word[pos:]
        elif op == "delete" and len(word) > 1:
            pos = random.randint(0, len(word) - 1)
            typo = word[:pos] + word[pos+1:]
        elif op == "remove_tone" and any(c in vowels for c in word):
            pos = random.choice([i for i, c in enumerate(word) if c in vowels])
            typo = word[:pos] + remove_tone(word[pos]) + word[pos+1:]
        elif op == "wrong_tone" and any(c in vowels for c in word):
            pos = random.choice([i for i, c in enumerate(word) if c in vowels])
            typo = word[:pos] + random_tone(word[pos]) + word[pos+1:]
        elif op == "transpose" and len(word) > 1:
            pos = random.randint(0, len(word) - 2)
            typo = word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
        elif op == "confused_initial":
            confused = confused_initials(word)
            if confused:
                typo = random.choice(confused)
        if typo and typo != word and typo not in typo_list:
            typo_list.append(typo)
    return typo_list

In [2]:
# Đọc danh sách từ
with open("vietnamese_only_tokens_sorted.txt", "r", encoding="utf-8") as f:
    words = [line.strip() for line in f if line.strip()]

# Sinh lỗi cho từng từ và lưu ra file
with open("typos_for_all_words.txt", "w", encoding="utf-8") as f:
    for word in words:
        typos = generate_typos(word, num_typos=50)  # Số lỗi mỗi từ, có thể thay đổi
        for typo in typos:
            f.write(f"{word}\t{typo}\n")

In [3]:
import re

def split_sentences(text):
    # Câu kết thúc bằng . ? !
    return re.split(r'(?<=[.?!])\s+', text)

with open("raw_articles.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

sentences = split_sentences(full_text)
print("Số câu:", len(sentences))
print("Ví dụ:", sentences[:3])

Số câu: 1588
Ví dụ: ['\nHà NộiBùi Lam Anh, 23 tuổi, bị tạm giữ hình sự để làm rõ cáo buộc chửi bới, quật ngã cảnh sát giao thông khi bị dừng xe kiểm tra.', 'TP HCM38 năm trước, bác sĩ Trần Thành Trai thực hiện ca mổ tách cặp dính liền Song - Pha, sau đó hai bé được một tỷ phú nhận con nuôi đưa sang Pháp và chưa một lần về lại quê hương.', 'Bộ tư lệnh không quân Ukraine cho biết Nga hôm nay tiến hành cuộc tập kích hiệp đồng nhằm vào nước này bằng 3 tên lửa đạn đạo Iskander-M, 4 tên lửa hành trình Kh-101 và Iskander-K, cùng 472 máy bay không người lái (UAV) tự sát Geran-2 và phi cơ mồi nhử.']


In [4]:
from random import choice
#from generate_typos import generate_typos  # dùng hàm của bạn

def introduce_typos_to_sentence(sentence, max_typos=1):
    words = sentence.split()
    indices = list(range(len(words)))
    typo_words = words.copy()
    typo_count = 0

    for _ in range(max_typos):
        idx = choice(indices)
        typos = generate_typos(words[idx], num_typos=3)
        if typos:
            typo_words[idx] = choice(typos)
            typo_count += 1

    if typo_count == 0:
        return None  # không sinh lỗi được
    return ' '.join(typo_words)

# Test
example = sentences[11]
typo_sent = introduce_typos_to_sentence(example)
print("Gốc:", example)
print("Lỗi:", typo_sent)

Gốc: Một năm trước, Nga triển khai 30 UAV tấn công Ukraine trong một đêm là điều hiếm thấy.
Lỗi: Một năm trước, Nga triển khai 30 UAV tấn công Ukraine trong một đm là điều hiếm thấy.


In [5]:
import re
from underthesea import word_tokenize

# Tải từ điển tiếng Việt (chứa từ thường)
with open("vietnamese_only_tokens_sorted.txt", "r", encoding="utf-8") as f:
    vietnamese_dict = set([line.strip() for line in f if line.strip()])

# Hàm loại bỏ dấu câu và chuẩn hóa từ để kiểm tra
def is_valid_word(word):
    # Bỏ ký tự không phải chữ cái (ví dụ: dấu chấm, phẩy)
    word = re.sub(r'[^\wàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữ'
                  r'ỳýỵỷỹđ]', '', word.lower())
    return word in vietnamese_dict

# Hàm phát hiện từ sai
def detect_misspelled_words(text, dictionary):
    tokens = word_tokenize(text, format="text").split()
    misspelled = []
    for i, word in enumerate(tokens):
        if not is_valid_word(word):
            misspelled.append((i, word))
    return tokens, misspelled

# Ví dụ sử dụng
input_text = "Tôi đag ăn com trưa ở nhà hang."
tokens, misspelled = detect_misspelled_words(input_text, vietnamese_dict)

print("Các từ sai:", misspelled)

Các từ sai: [(1, 'đag'), (3, 'com'), (7, 'hang'), (8, '.')]


In [6]:
from gensim.models import Word2Vec
from underthesea import word_tokenize

# Đọc dữ liệu
with open("raw_articles.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Tách câu → tách từ → thành tập train
sentences = raw_text.split('\n')  # hoặc dùng split_sentences()
tokenized_sentences = [word_tokenize(sentence, format="text").split() for sentence in sentences if sentence.strip()]

# Huấn luyện Word2Vec
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=2, workers=4)

# Lưu lại model
w2v_model.save("word2vec_vietnamese.model")


In [7]:
from difflib import get_close_matches

# Hàm gợi ý sửa từ sai
def suggest_correction(word, model, dictionary, topn=5):
    # 1. Tìm từ gần đúng trong từ điển dựa trên khoảng cách ký tự
    candidates = get_close_matches(word, dictionary, n=20, cutoff=0.7)
    
    # 2. Tìm trong số đó những từ có vector trong mô hình
    valid_candidates = [w for w in candidates if w in model.wv]

    # 3. Chọn top-n từ gần nhất trong không gian ngữ nghĩa
    suggestions = []
    for w in valid_candidates:
        sim = model.wv.similarity(word, w) if word in model.wv else 0
        suggestions.append((w, sim))
    
    suggestions.sort(key=lambda x: x[1], reverse=True)
    return [w for w, _ in suggestions[:topn]]

In [8]:
w2v_model = Word2Vec.load("word2vec_vietnamese.model")
suggestion = suggest_correction("đag", w2v_model, vietnamese_dict)
print("Gợi ý sửa:", suggestion)

Gợi ý sửa: ['đang']


In [10]:
import random
from underthesea import word_tokenize

# Danh sách câu đúng từ tập crawl
with open("raw_articles.txt", "r", encoding="utf-8") as f:
    raw_sentences = [line.strip() for line in f if line.strip()]

# Hàm tạo lỗi đơn giản bằng hoán đổi ký tự
def introduce_typos(sentence, typo_prob=0.2):
    def corrupt(word):
        if random.random() < typo_prob and len(word) > 3:
            pos = random.randint(0, len(word) - 2)
            return word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
        return word

    words = word_tokenize(sentence, format="text").split()
    corrupted = [corrupt(w) for w in words]
    return ' '.join(corrupted)

# Tạo dữ liệu lỗi
pairs = []
for s in raw_sentences:
    corrupted = introduce_typos(s)
    if corrupted != s:
        pairs.append((corrupted, s))

print("Ví dụ:")
print("Sai:", pairs[0][0])
print("Đúng:", pairs[0][1])

Ví dụ:
Sai: Hà_NộiBùi Lam_Anh , 23 tuổi , bị tạm giữ hình_sự để làm rõ cáo_buộc chửi_bới , uqật ngã cảnh_sát giao_thông khi bị dnừg xe kiểm_tra .
Đúng: Hà NộiBùi Lam Anh, 23 tuổi, bị tạm giữ hình sự để làm rõ cáo buộc chửi bới, quật ngã cảnh sát giao thông khi bị dừng xe kiểm tra.


In [51]:
pairs = [
    ("Toi an com", "tôi ăn cơm"),
    ("Toi hoc bai", "tôi học bài"),
    ("Toi di ngu", "tôi đi ngủ"),
    ("Toi uong nuoc", "tôi uống nước"),
    ("Toi di hoc", "tôi đi học"),
]


In [52]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

input_texts = [x[0] for x in pairs]
target_texts = ['<start> ' + x[1] + ' <end>' for x in pairs]

tokenizer = Tokenizer(filters='', lower=True, oov_token='<unk>')
tokenizer.fit_on_texts(input_texts + target_texts)

input_tensor = pad_sequences(tokenizer.texts_to_sequences(input_texts), padding='post')
target_tensor = pad_sequences(tokenizer.texts_to_sequences(target_texts), padding='post')

vocab_size = len(tokenizer.word_index) + 1
max_input_len = input_tensor.shape[1]
max_target_len = target_tensor.shape[1]

print("Vocab size:", vocab_size)

Vocab size: 22


In [53]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

embedding_dim = 128
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                ┃ Output Shape            ┃        Param # ┃ Connected to            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer) │ (None, None)            │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ input_layer_14 (InputLayer) │ (None, None)            │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ embedding_7 (Embedding)     │ (None, None, 128)       │          2,816 │ input_layer_13[0][0]    │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ embedding_8 (Embedding)     │ (None, None, 128)       │          2,816 │ input_layer_14[0][0]    │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ lstm_4 (LSTM)               │ [(None, 256), (None,    │        394,240 │ embedding_7[0][0]       │
│                             │ 256), (None, 256)]      │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ lstm_5 (LSTM)               │ [(None, None, 256),     │        394,240 │ embedding_8[0][0],      │
│                             │ (None, 256), (None,     │                │ lstm_4[0][1],           │
│                             │ 256)]                   │                │ lstm_4[0][2]            │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_2 (Dense)             │ (None, None, 22)        │          5,654 │ lstm_5[0][0]            │
└─────────────────────────────┴─────────────────────────┴────────────────┴─────────────────────────┘

 Total params: 799,766 (3.05 MB)

 Trainable params: 799,766 (3.05 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
target_tensor_in = target_tensor[:, :-1]
target_tensor_out = target_tensor[:, 1:]

target_tensor_out = np.expand_dims(target_tensor_out, -1)

model.fit([input_tensor, target_tensor_in], target_tensor_out, batch_size=2, epochs=100)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2031 - loss: 3.0890
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5000 - loss: 3.0369
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5000 - loss: 2.9622
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5000 - loss: 2.8252
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5000 - loss: 2.5677
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5000 - loss: 2.1700
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5000 - loss: 2.2016
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5000 - loss: 1.9098 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5000 - loss: 1.8182
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5000 - loss: 1.6032
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5000 - loss: 1.5186
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5250 - lo

In [56]:
# Encoder model để dự đoán trạng thái từ câu đầu vào
encoder_model = Model(encoder_inputs, [state_h, state_c])

In [60]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# Các input cho decoder khi suy diễn
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Input chính cho decoder
decoder_inputs = Input(shape=(1,))  # chỉ 1 bước thời gian tại mỗi vòng lặp

# Lớp embedding (phải giống với training)
decoder_embedding_layer = Embedding(vocab_size, embedding_dim)
dec_emb = decoder_embedding_layer(decoder_inputs)

# LSTM trong chế độ suy diễn
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb, initial_state=decoder_states_inputs)

# Lớp Dense để dự đoán từ tiếp theo
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Trả về cả output và state để dùng trong bước tiếp theo
decoder_states2 = [state_h2, state_c2]

# Xây dựng mô hình decoder
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [61]:
def decode_sequence(input_sentence):
    input_seq = tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_input_len, padding='post')

    # Bước 1: mã hóa input để lấy trạng thái ban đầu
    states_value = encoder_model.predict(input_seq)

    # Bắt đầu với token <start>
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_word_index.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence) > max_target_len:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

            # Cập nhật target_seq và trạng thái tiếp theo
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index
            states_value = [h, c]

    return ' '.join(decoded_sentence)

In [62]:
print("Sửa:", decode_sequence("Toi an com"))
print("Sửa:", decode_sequence("Toi ún nuoc"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Sửa: tôi ăn cơm
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Sửa: tôi uống nước
